In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from decpomdp import DecPOMDP
import Classes
from constant import Constants
import time
import os



In [2]:
file_name = "dectiger"
planning_horizon = 1
num_iterations = 1
sota_ = False


In [3]:
#import problem
problem = DecPOMDP(file_name, 1,horizon=planning_horizon)
# constant = Constants(problem)
# utilities = Utilities(constant)
Classes.set_problem(problem)

# print(f"game of initiated with SOTA set to = {sota_} with horizon {planning_horizon}")
print(f"game size :\n\t|S| = {len(problem.states)}")
print(f"\t|Z| = {problem.num_joint_observations}\n\t|U| = {problem.num_joint_actions} with |U_i| = {problem.num_actions[0]}")
print(f"intiial_belief : {problem.b0}")


print(f"{problem.num_actions[0]},{problem.num_actions[0]}")

problem set to dectiger
game size :
	|S| = 2
	|Z| = 4
	|U| = 9 with |U_i| = 3
intiial_belief : [0.5 0.5]
3,3


In [4]:
# solve
def SOLVE(game):
    start_time = time.time()
    policy = game.solve(num_iterations,0.9)
    end_time = time.time()
    solve_time = end_time - start_time
    value_fn = game.value_function
    return policy,solve_time,value_fn

In [5]:
def initialize_database():
    database = {"gametype":[],
                "SOTA" : [],
                "horizon": [],
                    "num_iterations" : [],
                    "average_time" : [],
                    "number_of_beliefs" : [],
                    "ave_leader_value_b0":[],
                    "ave_follower_value_b0":[]
                    # "density" = []
                    # "gap":[]
                   
                    }
    return database

def add_to_database(database,horizon,game_type,num_iterations,average_time,num_beliefs,V0_B0,V1_B0,SOTA):
    sota = {True:"State of the Art" , False:"Stackelberg"}
    database["gametype"].append(game_type)
    database["horizon"].append(horizon)
    database["SOTA"].append(sota[SOTA])
    database["num_iterations"].append(num_iterations)
    database["average_time"].append(average_time)
    database["number_of_beliefs"].append(num_beliefs)
    database["ave_leader_value_b0"].append(V0_B0)
    database["ave_follower_value_b0"].append(V1_B0)
    # database["gap"].append(abs(V0_B0-V1_B0))
    # database["density"].append(density)
    return


In [6]:
database = initialize_database()
for gametype in ["cooperative","zerosum","stackelberg"]:
    for horizon in range(1,planning_horizon+1):
        problem = DecPOMDP(file_name,1,horizon=planning_horizon)
        Classes.set_problem(problem)
        for sota_ in [False,True]:
            print(f"\n============= {gametype} GAME WITH HORIZON {horizon} , SOTA {sota_} ===========")
          
            game = Classes.PBVI(problem=problem,horizon=horizon,density=0.1,gametype=gametype,sota=sota_)
            policy, time_ , value_fn = SOLVE(game)
            num_beliefs = game.belief_space.belief_size()
            value0,value1= value_fn.get_values_initial_belief()
            add_to_database(database,horizon,gametype,2,time_,num_beliefs,value0,value1,sota_)
#POLICY PRINTING: 
# print("\nLEADER POLICY\n")
# policy[0].print_trees()
# print("\nFOLLOWER POLICY\n")
# policy[1].print_trees()        



database = pd.DataFrame(database)
path = "Results/"
file_name = f"{file_name}_{horizon}_experiment_results.csv"
database.to_csv(file_name, index=False)



problem set to dectiger

============= cooperative GAME WITH HORIZON 1 , SOTA False ===========
	belief expansion done
iteration : 1


	backup at timestep 1 done
	backup at timestep 0 done
	backward induction done

============= cooperative GAME WITH HORIZON 1 , SOTA True ===========
	belief expansion done
iteration : 1
	backup at timestep 1 done
	backup at timestep 0 done
	backward induction done
problem set to dectiger

============= zerosum GAME WITH HORIZON 1 , SOTA False ===========
	belief expansion done
iteration : 1
	backup at timestep 1 done
	backup at timestep 0 done
	backward induction done

============= zerosum GAME WITH HORIZON 1 , SOTA True ===========
	belief expansion done
iteration : 1
	backup at timestep 1 done
	backup at timestep 0 done
	backward induction done
problem set to dectiger

============= stackelberg GAME WITH HORIZON 1 , SOTA False ===========
	belief expansion done
iteration : 1
	backup at timestep 1 done
	backup at timestep 0 done
	backward induction done

============= stackelberg GAME WITH HORIZON 1 , SOTA True ===========
	belief expansion done
iteration : 1
	backup at timestep 1 d

In [7]:
database

,gametype,SOTA,horizon,num_iterations,average_time,number_of_beliefs,ave_leader_value_b0,ave_follower_value_b0
0,cooperative,Stackelberg,1,2,0.189921,4,[10.815000000000001],[10.815000000000001]
1,cooperative,State of the Art,1,2,0.005508,4,[10.815000000000001],[10.815000000000001]
2,zerosum,Stackelberg,1,2,0.083990,4,[-88.22274983811785],[88.22274983811785]
3,zerosum,State of the Art,1,2,0.155922,4,[-88.22274983811785],[-88.22274983811785]
4,stackelberg,Stackelberg,1,2,0.083272,4,[3.468480039267017],[-78.47541939354277]
5,stackelberg,State of the Art,1,2,0.060585,4,[2.4107554144557923],[-52.611111111111114]


game.value_function

In [8]:
for i in game.value_function.vector_sets[0]:
    print(i.get_value(game.belief_space.get_inital_belief()))

(2.4107554144557923, -52.611111111111114)


In [9]:
print(f"RESULTS FOR {file_name}:\n")

RESULTS FOR dectiger_1_experiment_results.csv:

